# Deep Research with AutoGen

This notebook implements a multi-agent deep research system using AutoGen. It creates a team of specialized agents that collaborate to perform in-depth research on a given topic.

In [15]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
import timeit
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.tools import TeamTool
# or import
from autogen_core.models import ModelInfo
import sys

sys.path.append(os.path.abspath(".."))
# Token usage tracking
from utilities.token_counting import (
    wrap_chat_client_for_tokens,
    print_token_usage_with_estimate,
    log_token_usage,
)

from utilities import (
    create_chat_completion_client,
    current_datetime_utc_tool,
    adjust_properties_for_fixed_response,
    get_tavily_search_tool,
    get_attributes,
    load_model_config,
)

#### Note: You need a tavily API key to use the web search tool!

In [16]:
model = load_model_config() 

# Check if web search is possible (i.e., if API key is set)
ALLOW_WEB_SEARCH = os.getenv("TAVILY_API_KEY", False)

Using model: gpt-4.1-nano type: openai


In [17]:
# Prepare tools
basic_tools = {current_datetime_utc_tool}
web_search_tools = set()
if ALLOW_WEB_SEARCH:
    # web_search_tool = [ await get_rag_web_browser_tool() ]
    web_search_tools = { await get_tavily_search_tool() }

Set the research topic
----------------------

In [18]:
# topic = "Impacts of Model Context Protocol on Agentic AI"
# topic = "What is Fraunhofer MEVIS?"
# topic = "What day and time is it in Berlin, Germany?"
# topic = "What is the wheather like in Berlin, Germany currently?"
topic = "What has Fraunhofer MEVIS done in the context of thoracic imaging?"

Select Model and Agent Configuration
-----------------------------

In [19]:

agentConfig = {
    "Researcher": dict(system_message="Gather and summarize factual info, add important web references.", tools=web_search_tools), 
    "FactChecker": dict(system_message="Use web search to verify facts and citations from the Researcher's response. For each found error, give a correct version and the web reference for verification.", tools=web_search_tools), 
    "Revisor": dict(system_message="Use the fact checker's results to revise the Researcher's report. Only if there were no fact issues, just repeat the Researcher's response. In both cases, add the fact checker's citations."),
    "Critic": dict(system_message="Critique clarity and logic."),
    "Summarizer": dict(system_message="Apply critic result to generate an improved response to the initial user question."),# Include the most important references."),
    "Editor": dict(system_message="Do some language polishing on the previous result to generate a final response to the initial question. Signal APPROVED when done."),
}

# For testing, we can fix an agent's response to a known answer
# adjust_properties_for_fixed_response(agentConfig["Researcher"], "Fraunhofer MEVIS is a research institute in Frankfurt for biological weapons. It was founded in 1995. See https://www.mefis.fraunhofer.de/en.html for more information.")

In [20]:
# Now create model client with token tracking
model_client = wrap_chat_client_for_tokens(create_chat_completion_client(model))

agents = []  # start with a user proxy for the researcher

for agent_type, props in agentConfig.items():
    p = props.copy()
    p.setdefault("tools", set()).update(basic_tools)
    p.setdefault("reflect_on_tool_use", True)
    agents.append(
        AssistantAgent(
            name=agent_type,
            **p,
            model_client=model_client,
        )
    )

In [21]:
# Set up termination conditions
max_msgs = MaxMessageTermination(max_messages=20)
text_term = TextMentionTermination(text="APPROVED", sources=["Editor"])
termination = max_msgs | text_term

# Create the team
team = RoundRobinGroupChat(
    participants=agents,
    termination_condition=termination,
    max_turns=len(agents) # we don't really want it to go round
)

In [22]:
# Create the host agent with deep dive capabilities and the host agent
deepdive_tool = TeamTool(team=team, name="DeepDive", description="Collaborative multi-agent deep dive")
host = AssistantAgent(
    name="Host",
    model_client=model_client,
    tools=[deepdive_tool],
    system_message="You have access to a DeepDive tool for in-depth research."
)

Start the Deep Dive
===================

In [ ]:
async def run_deepdive(topic: str):
    start_time_s = timeit.default_timer()
    result = await team.run(task=f"Deep dive on: {topic}")
    # Display token usage after the deep dive
    print_token_usage_with_estimate(model_client, getattr(result, 'messages', None), agentConfig.keys(), model_info=model)
    elapsed = timeit.default_timer()-start_time_s
    print(f"🔍 DeepDive result (took {elapsed:.2f} sec.) resulted in {len(result.messages)} messages.")
    return result  # Return the result for further exploration

# Run the deep dive
result = await run_deepdive(topic)

Estimated token usage -> prompt: 2633, completion: 4160, total: 6793 (method=tiktoken)
Approximate cost for this run: $0.0010 USD (input: $0.0001, output: $0.0008)
🔍 DeepDive result (took 25.31 sec.) resulted in 11 messages.


Explore Results
===============

In [31]:
print("*** Stop reason:", result.stop_reason, "\n")

# Display the full result structure
# print("Result type:", type(result))
# print("\nResult attributes:")
# print(get_attributes(result))
# print("\nMessage attributes:")
# print(get_attributes(result.messages[-1]))

print("*** Message Sequence:")
for i, m in enumerate(result.messages):  
    print(f"\n----------------------------------------------------------------")
    print(f"MESSAGE {i} [{m.source}]:\n{m.content}") # type: ignore

*** Stop reason: Text 'APPROVED' mentioned 

*** Message Sequence:

----------------------------------------------------------------
MESSAGE 0 [user]:
Deep dive on: What has Fraunhofer MEVIS done in the context of thoracic imaging?

----------------------------------------------------------------
MESSAGE 1 [Researcher]:
[FunctionCall(id='call_OxpK664pFO6h9PB8j3CMzZaO', arguments='{"query": "Fraunhofer MEVIS thoracic imaging"}', name='tavily_search'), FunctionCall(id='call_XhlhSiWXj0NmjKMai1bqFk8f', arguments='{"query": "Fraunhofer MEVIS contributions to thoracic imaging"}', name='tavily_search')]

----------------------------------------------------------------
MESSAGE 2 [Researcher]:
[FunctionExecutionResult(content='Search Query: Fraunhofer MEVIS thoracic imaging\nResults:\n1. Solutions for Cardiac Imaging - Fraunhofer MEVIS\nURL: https://www.mevis.fraunhofer.de/en/solutionpages/solutions-for-cardiac-imaging.html\nSnippet: Fraunhofer MEVIS develops software solutions for multimodal c

In [33]:
from autogen_agentchat.messages import BaseTextChatMessage

assert isinstance(result.messages[-1], BaseTextChatMessage)
last_message = result.messages[-1].content

print("\nFINAL RESULT:\n", last_message)  # Print the content of the last message


FINAL RESULT:
 Fraunhofer MEVIS has made substantial, validated contributions to thoracic imaging, primarily through innovative software solutions for automated, AI-assisted analysis of lung CT and MRI scans. Their advanced segmentation techniques for lungs and pulmonary lobes have been validated via challenges like LOLA11, demonstrating their robustness. Additionally, they have developed rapid and precise lung image registration algorithms that enable detailed longitudinal comparisons with sub-millimeter accuracy, supporting accurate diagnosis and treatment planning. Their active engagement in prominent scientific conferences such as MICCAI underscores their leadership in thoracic imaging research. Overall, their work significantly enhances diagnostic accuracy and clinical decision-making in thoracic health.

**References for verification:**
- [Fraunhofer MEVIS MICCAI participation](https://www.mevis.fraunhofer.de/en/press-and-scicom/institute-news/2020/fraunhofer-mevis-at-virtual-mi

In [28]:
# Optional: append token usage (estimated or native) to a CSV log for tracking across runs
# log_token_usage imported from token_counting
log_token_usage(topic, model_client, getattr(result, 'messages', None), agentConfig.keys(), model_info=model)

Logged token usage to token_usage_log.csv: {'timestamp': '2025-09-24T10:26:26.148877+00:00', 'topic': 'What has Fraunhofer MEVIS done in the context of thoracic imaging?', 'prompt': 2633, 'completion': 4160, 'total': 6793, 'method': 'tiktoken', 'cost_usd': 0.000964, 'input_cost_usd': 0.000132, 'output_cost_usd': 0.000832}


{'timestamp': '2025-09-24T10:26:26.148877+00:00',
 'topic': 'What has Fraunhofer MEVIS done in the context of thoracic imaging?',
 'prompt': 2633,
 'completion': 4160,
 'total': 6793,
 'method': 'tiktoken',
 'cost_usd': 0.000964,
 'input_cost_usd': 0.000132,
 'output_cost_usd': 0.000832}